# Scrapping on WoS to get dismissed and undismissed authors

In [62]:
import time
from scraper_WoS import WosClient
import pandas as pd
from unicode_tr.extras import slugify

In [63]:
def get_author_names(path_text_file):
#getting all the names 
    searchnames = []
    i=0
    for line in open(path_text_file,'r'):
        QueryString = line.strip()
        QueryString = "AU="+line.split(',')[0].split()[-1]+' '+line.split(',')[0].split()[0]
        searchnames.append(QueryString)
    return  searchnames, QueryString

In [64]:
def searching_co_authors_WoS(searchnames):
    soap = WosClient(lite=True)
    soap.connect()
    names = []
    lens = []
    co_aut = []
    i=0
    for i in range(len(searchnames)):
        #line = SearchNames.SearchNames[i]
        QueryString = searchnames[i]
        #QueryString = line
        print(QueryString)
        if i == 2400: # WOS constraints that each ID can only search for 2500 times. Just set smaller counts to renew the ID.
            soap = WosClient(lite=True)
            soap.connect()
        results = soap.search(QueryString)
        #getting number of name of co-authors
        if (results.recordsFound>0):
            authors = []
            for j in range (len(results.records)):
                for k in range (len(results.records[j].authors)):
                    for l in range (len(results.records[j].authors[k].value)):
                        co_authors=results.records[j].authors[k].value[l]
                        authors.append(co_authors)
                        print('co_author',co_authors)
        print(results.recordsFound)
        names.append(QueryString.split('=')[1])
        lens.append(results.recordsFound)
        co_aut.append(authors)
        time.sleep(0.5) # WOS constraints that you can only search twice per second
        i=i+1        
    return names, lens, co_aut

In [65]:
#a python module to make unicode strings work as expected for turkish chars. solves the turkish "İ" problem

# have to do pip install unicode_tr
def translate_names(names):
    names_en = []
    for i in range(len(names)):
        name_en = slugify(names[i])
        name_en = name_en.replace('-',' ').title()
        names_en.append(name_en)
    return names_en


In [66]:
#formatting co authors column to match with authors
#for i in range(df.shape[0]):
#    for j in range(len(df.CoAuthors.iloc[i])):
#        a=df.CoAuthors.iloc[i]
#        a[j]=a[j].replace(","," ")
#        df.CoAuthors.iloc[i] = a
#df

In [67]:
def count_nb_coauthors(co_aut):
    cnt_coauthors=0
    for l in co_aut:
        cnt_coauthors += len(l)
    print('Totally ' + str(cnt_coauthors) + ' guys were downloaded as coauthors')
    return cnt_coauthors

In [68]:
def save_csv(list1,list2):
    dismissed_df = pd.DataFrame()
    dismissed_df['AU']=list(list1)
    dismissed_df.to_csv('../data/dismissed.csv')
    undismissed_df = pd.DataFrame()
    undismissed_df['AU']=list(list2)
    undismissed_df.to_csv('../data/undismissed.csv')


In [69]:
searchnames, QueryString = get_author_names('../Data/Dismiss_Acad_List.txt')
names,lens,co_aut = searching_co_authors_WoS(searchnames)
names_en=translate_names(names)

#dataset with dimissed people with their correspondant coauthors and number of papers
df = pd.DataFrame([names,lens,co_aut]).T
df.columns=['Author','NoPapers','CoAuthors']
df

nb_coauthors = count_nb_coauthors(co_aut)

coauthors = set()
for l in co_aut:
    for author in l:
        author = author.replace(',', '')
        coauthors.add(author)

dismissed = set(names_en)
print('Totally we have ' + str(len(dismissed)) + ' dismissed and ' + str(len(coauthors)) + ' unique coauthors')

undismissed = coauthors - dismissed
print('So, we have ' + str(len(undismissed)) + ' undismissed guys')

save_csv(dismissed,undismissed)

1
2
Authenticated (SID: 6A3XSwncToPNJPzPCYv)
AU=AYDIN AYDIN
co_author Kul, Sibel
co_author Aydin, Aydin
co_author Dinc, Hasan
co_author Erduran, Erol
co_author Aydin, Leyla
co_author Aydin, Aydin
co_author Bulbul, Baris
co_author Ozkalkan, Beliz
co_author Ozaydin, Cansu
co_author Inaltekin, Ezgi
co_author Aydin, Aydin
co_author Aksoy, Hamit Zafer
co_author Reis, Abdulkadir
co_author Ozdemir, Feyyaz
co_author Zengin, Ahmet Y.
co_author Topkan, Erkan
co_author Aydin, Aydin
co_author Reis, A. Kadir
4
AU=GİLGİL ERDAL
co_author Melikoglu, Meltem Alkan
co_author Kocabas, Hilal
co_author Sezer, Ilhan
co_author Akdag, Ali
co_author Gilgil, Erdal
co_author Butun, Bulent
co_author Dundar, Umit
co_author Kupesiz, Alphan
co_author Ozdem, Sehahat
co_author Gilgil, Erdal
co_author Tuncer, Tiraje
co_author Yesilipek, Akif
co_author Gultekin, Meral
co_author Gilgil, Erdal
co_author Kacar, Cahit
co_author Tuncer, Tiraje
co_author Tuncer, Tiraje
co_author Gilgil, Erdal
co_author Kacar, Cahit
co_author K

AU=ŞENGÜL ABDULLAH
co_author Emsley, Robin
co_author Hargarter, Ludger
co_author Bergmans, Paul
co_author Uglesic, Boran
co_author Sengul, Abdullah Cem
co_author Petralia, Antonino
co_author Khannanova, Angelina
co_author Cherubin, Pierre
co_author Schreiner, Andreas
co_author Karaoglan, Mustafa
co_author Tutar, Suleyman
co_author Sengul, Abdullah Murat
co_author Tekeli, Hakan
co_author Togrol, Rifat Erdem
co_author Sengul, Abdullah
3
AU=DEMİR ABDULLAH
co_author Ulucanlar, Haluk
co_author Agac, Aybala
co_author Kutun, Suat
co_author Demir, Abdullah
co_author Yukruk, Fusun Ardis
co_author Cetin, Abdullah
co_author Topcu, Ilker Bekir
co_author Demir, Abdullah
co_author Uysal, Tancan
co_author Basciftci, Faruk Ayhan
co_author Sener, Yagmur
co_author Botsali, Murat Selim
co_author Demir, Abdullah
co_author Bekir Topcu, Ilker
co_author Demir, Abdullah
co_author Sener, Yagmur
co_author Uysal, Tancan
co_author Basciftci, Farulk Ayhan
co_author Demir, Abdullah
co_author Botsali, Murat Selim
54

AU=ÖZTÜRK ABDURAHMAN
0
AU=ABAKAY ABDURRAHMAN
co_author Tanrikulu, A. Cetin
co_author Hosoglu, Salih
co_author Ozekinci, Tuncer
co_author Abakay, Abdurrahman
co_author Gurkan, Fuat
co_author Tanrikulu, Abdullah C.
co_author Senyigit, Abdulrahman
co_author Dagli, Canan E.
co_author Babayigit, Cenk
co_author Abakay, Abdurrahman
co_author Kaplan, Muhammet Ali
co_author Duzkopru, Yakup
co_author Ulku, Refik
co_author Tanrikulu, Cetin
co_author Kucukoner, Mehmet
co_author Onat, Serdar
co_author Abakay, Abdurrahman
co_author Urakci, Zuhat
co_author Meteroglu, Fatih
co_author Oruc, Zeynep
co_author Isikdogan, Abdurrahman
co_author Sen, Hadice Selimoglu
co_author Abakay, Ozlem
co_author Tanrikulu, Abdullah Cetin
co_author Sezgi, Cengizhan
co_author Taylan, Mahsuk
co_author Abakay, Abdurrahman
co_author Kaya, Halide
co_author Senyigit, Abdurrahman
co_author Sen, Hadice Selimoglu
co_author Kaplan, Ibrahim
co_author Abakay, Ozlem
co_author Sezgi, Cengizhan
co_author Yilmaz, Sureyya
co_author TayIa

AU=UĞU ABDÜLHADİ
0
AU=ÖZDENİZ ABDÜLHADİ
co_author Ozdeniz, Abdulhadi. H.
co_author Kelebek, Sadan
1
AU=IŞIK ABDÜLKADİR
0
AU=KOÇER ABDÜLKADİR
co_author Kocer, Abdulkadir
co_author Dikici, Sueber
co_author Atakay, Selcuk
co_author Okuyucu, Sefika
co_author Kocer, Emel
co_author Bilge, Cevdet
co_author Cagrici, Sultan
co_author Tutkan, Hava
co_author Kocer, Abdulkadir
co_author Kocer, Abdulkadir
co_author Degirmenci, Yildiz
co_author Yueksel, Harun
co_author Eryilmaz, Mehmet
co_author Cesur, Mustafa
co_author Bayram, Fahri
co_author Temel, Mehtap Akcil
co_author Ozkaya, Mesut
co_author Kocer, Abdulkadir
co_author Ertorer, Melek Eda
co_author Koc, Filiz
co_author Kaya, Ahmet
co_author Gullu, Sevim
co_author Domac, Fusun M.
co_author Kocer, Abdulkadir
co_author Tanidir, Remzi
36
AU=ÖZTÜRK ABDÜLKADİR
co_author Koruk, Serdar
co_author Ozturk, Abdulkadir
co_author Kara, Ahmet
co_author Ozturk, Abdulkadir
co_author Ozturk, Abdulkadir
co_author Mutlu, Tansu
3
AU=KARSLI ABDÜRRAHİM
0
AU=ARTIŞ ABDÜ

AU=TOK ADEM
co_author Muslumanoglu, Ahmet Yaser
co_author Karadag, Mert Ali
co_author Tefekli, Ahmet Hamdi
co_author Altunrende, Fatih
co_author Tok, Adem
co_author Berberoglu, Yalcin
co_author Karakas, Huseyin Bugra
co_author Cicekbilek, Izzet
co_author Tok, Adem
co_author Aliskan, Tamer
co_author Akduman, Bulent
co_author Kocaaslan, Ramazan
co_author Kayali, Yunus
co_author Tok, Adem
co_author Tepeler, Abdulkadir
co_author Tok, Adem
co_author Akbulut, Fatih
co_author Buldu, Ibrahim
co_author Karatag, Tuna
co_author Kucuktopcu, Onur
co_author Gurbuz, Gokhan
co_author Istanbulluoglu, Okan
co_author Armagan, Abdullah
co_author Tepeler, Abdulkadir
co_author Tasci, Ali Ihsan
co_author Erol, Bulent
co_author Tuncel, Altug
co_author Tok, Adem
co_author Hanci, Volkan
co_author Sari, Utku
co_author Sendogan, Furkan
co_author Budak, Salih
co_author Aydemir, Huseyin
co_author Amasyali, Akin Soner
co_author Yildirim, Asif
co_author Caskurlu, Turhan
16
AU=TOPKARA ADEM
co_author Oksuz, Mustafa
co_

...

AU=GÜL İSMAİL
co_author Gul, Ismail
co_author Yildirim, Mehmet
co_author Akinci, Cuma
co_author Doran, Ilhan
co_author Kilic, Hasan
co_author Gul, Ismail
co_author Demirel, Ramazan
co_author Kilicalp, Numan
co_author Sumerli, Mehdi
co_author Kilic, Hasan
co_author Gul, Ismail
co_author Akinci, Cuma
co_author Doran, Ilhan
co_author Kilic, Hasan
co_author Baytekin, Harun
co_author Gul, Ismail
co_author Staal, Jens
co_author Driege, Yasmine
co_author Haegman, Mira
co_author Borghi, Alice
co_author Hulpiau, Paco
co_author Lievens, Laurens
co_author Gul, Ismail Sahin
co_author Sundararaman, Srividhya
co_author Goncalves, Amanda
co_author Dhondt, Ineke
co_author Pinzon, Jorge H.
co_author Braeckman, Bart P.
co_author Technau, Ulrich
co_author Saeys, Yvan
co_author van Roy, Frans
co_author Beyaert, Rudi
15
AU=GÜLCÜ İSMAİL
0
AU=UZUN İSMAİL
co_author Uzun, Ismail
co_author Arisoy, Ebru
co_author Edizkan, Rifat
co_author Saraclar, Murat
co_author IEEE
co_author Uzun, Ismail
co_author Edizkan, Ri

AU=YILDIZ İSMAİL
co_author Kilic, Ayse
co_author Unuvar, Emin
co_author Yilmaz, Ceren
co_author Yildiz, Ismail
co_author Oguz, Fatma
co_author Sidal, Mujgan
co_author Ergun, Yilmaz
co_author Sagliker, Yahya
co_author Paylar, Nuray
co_author Aikimbaer, K.
co_author Yildiz, Ismail
co_author Sagliker, Yahya
co_author Uzel, Asli
co_author Uzel, Ilter
co_author Yildiz, Ismail
co_author Paylar, Nuray
co_author Halvaci, Ilker
co_author Ergun, Yilmaz
co_author Sagliker, Yahya
co_author Paylar, Nuray
co_author Yildiz, Ismail
co_author Sagliker, Yahya
co_author Acharya, Vidya
co_author Ling, Zhang
co_author Golea, Ovidia
co_author Sabry, Alaa
co_author Eyupoglu, Kamil
co_author Ookalkar, Dhananjay S.
co_author Tapiawala, Shruti
co_author Durugkar, Santosh
co_author Khetan, Prakash
co_author Capusa, Cristina
co_author Univar, Ramazan
co_author Yildiz, Ismail
co_author Cengiz, Kuddisi
co_author Bali, Musa
co_author Ozkaynak, Piril Sagliker
co_author Sagliker, Hasan Sabit
co_author Paylar, Nuray
co

AU=GÜÇLÜ ŞABAN
co_author Guclu, Saban
co_author Hayat, Ruestem
co_author Shorthouse, Joseph D.
co_author Tozlu, Goeksel
co_author Ak, Kibar
co_author Guclu, Saban
co_author Eken, Cafer
co_author Sekban, Reyhan
co_author Guclu, Saban
co_author Kavaz, Handan
co_author Guclu, Coskun
co_author Ozdemir, Isil
co_author Yildiz, Hilal
co_author Ercisli, Sezai
co_author Narmanlioglu, Haluk Kemal
co_author Guclu, Saban
co_author Akbulut, Mustafa
co_author Turkoglu, Zumrut
co_author Eken, Cafer
co_author Guclu, Saban
co_author Ak, Kibar
9
AU=TEZCAN ŞABAN
0
AU=ÇETİN ŞABAN
co_author Aggumus, Huseyin
co_author Cetin, Saban
co_author Kurt, Ilkay
co_author Paksoy, Mahmut
co_author Cetin, Saban
co_author Guclu, Rahmi
co_author Sezer, Semih
co_author IEEE
co_author Cetin, Saban
co_author Yildiz, Ali Suat
co_author Sivrioglu, Selim
co_author Zergeroglu, Erkan
co_author Cetin, Saban
co_author Paksoy, Mahmut
co_author Guclu, Rahmi
co_author Cetin, Saban
12
AU=AYDIN ŞADİ
0
AU=BARANOĞLU ŞAHİN
0
AU=BATMAZ ŞAK

AU=ŞİMŞEK ŞEREF
co_author Cetin, Ihsan
co_author Simsek, Seref
co_author Cetin, Ihsan
co_author Bulut, Hamdullah
co_author Simsek, Seref
co_author Simsek, Seref
co_author Gencoglan, Salih
co_author Yuksel, Tugba
co_author Aktas, Huseyin
co_author Simsek, Seref
co_author Cetin, Ihsan
co_author Cim, Abdullah
co_author Kaya, Savas
co_author Simsek, Seref
co_author Gencoglan, Salih
co_author Yuksel, Tugba
co_author Kaplan, Ibrahim
co_author Aktas, Huseyin
co_author Alaca, Rumeysa
20
AU=TUNA ŞERİF
0
AU=KAHRAMAN ŞERİF
co_author Alp, Aydin
co_author Kahraman, Serif
co_author Kahraman, Serif Samil
co_author Yildirim, Yavuz Selim
co_author Tugrul, Selhattin
co_author Ozturan, Orhan
co_author Kahraman, Serif Samil
co_author Arli, Cengiz
co_author Copoglu, Umit Sertan
co_author Kokacya, Mehmet Hanifi
co_author Colak, Sait
co_author Ekiz, Ozlem
co_author Kahraman, Serif Samil
co_author Sen, Bilge Bulbul
co_author Serarslan, Gamze
co_author Rifaioglu, Emine Nur
co_author Culha, Gulnaz
co_author Ozg

AU=DÜZGÜN ŞÜKRÜ
co_author Duzgun, Sukru Aydin
co_author Zeren, Sezgin
co_author Bayhan, Zulfu
co_author Zeren, Sezgin
co_author Ozbay, Isa
co_author Bayhan, Zulfu
co_author Kocak, Fatma Emel
co_author Ucar, Bercis Imge
co_author Kahraman, Cuneyt
co_author Duzgun, Sukru Aydin
co_author Kocak, Cengiz
co_author Duzgun, Sukru Aydin
co_author Yerlikaya, Azmi
co_author Zeren, Sezgin
co_author Bayhan, Zulfu
co_author Okur, Emrah
co_author Boyaci, Ihsan
co_author Bayhan, Zulfu
co_author Zeren, Sezgin
co_author Duzgun, Sukru Aydin
co_author Ucar, Bercis Imge
co_author Bayhan, Zulfu
co_author Zeren, Sezgin
co_author Kocak, Cengiz
co_author Kocak, Fatma Emel
co_author Duzgun, Sukru Aydin
co_author Algin, Mustafa Cem
co_author Taskoylu, Burcu Yapar
co_author Yaylak, Faik
10
AU=BOYLU ŞÜKRÜ
co_author Meydan, Nezih
co_author Barutca, Sabri
co_author Guney, Engin
co_author Boylu, Sukru
co_author Savk, Oner
co_author Culhaci, Nil
co_author Ayhan, Mediha
co_author Erpek, Hakan
co_author Tuncyurek, Pars
